 Deep Learning Begins:
 Assignment 1
 Task 1 Implement 2-layer MLP for MNIST dataset

Import the needed libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Load, concatenate, and then properly split dataset (70/30)

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
X = np.concatenate([x_train, x_test])
y = np.concatenate([y_train, y_test])

#then split using train_test_split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=(0.3))

11490434/11490434 [==============================] - 2s 0us/step


In [3]:
# Python starts with 0 so if label is 5 one-hot encoding will show it in 6th place

# Here we have a class number for each image
print("Class label of first image :", y_train[0])

# Convert to One-Hot Encoding
y_train = tf.keras.utils.to_categorical(y_train, 10) 
y_test = tf.keras.utils.to_categorical(y_test, 10)

print("After converting the output into a vector : ",y_train[0])

Class label of first image : 9
After converting the output into a vector :  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [4]:
# Inspect the image pixel value
x_train[0].min(), x_train[0].max()

(0, 255)

In [5]:
# Normalize the data between [0,1]

x_train = x_train/255.0
x_test = x_test/255.0

x_train[0].min(), x_train[0].max()

(0.0, 1.0)

In [6]:
# Check the shapes of the data

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((49000, 28, 28), (21000, 28, 28), (49000, 10), (21000, 10))

In [7]:
# Reshape so each 'dataset' above is flattened into one vector 

x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)


x_train.shape, x_test.shape, y_train.shape, y_test.shape

((49000, 784), (21000, 784), (49000, 10), (21000, 10))

In [8]:
# Build the model

# Define the optimizer
adam_opt = tf.keras.optimizers.Adam(learning_rate=0.0001,
                                    beta_1=0.9,
                                    beta_2=0.999)
# Define the loss
loss_ = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Define the number of units per layer
feature_size = 784
unit_size = 100
n_hidden_layers = 2

def build_mlp_model(feature_size, n_hidden_layers, unit_size, opt_, loss_):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(unit_size, activation=tf.nn.relu, input_shape=(feature_size,)),
        tf.keras.layers.Dense(unit_size, activation=tf.nn.relu),
        tf.keras.layers.Dense(10)
    ])
    
    model.compile(optimizer=opt_, 
                  loss=loss_,
                  metrics=['accuracy'])
    
    return model


mlp = build_mlp_model(feature_size, n_hidden_layers, unit_size, adam_opt, loss_)

mlp.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Start training the model

batch_size=32
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = mlp.fit(x_train, y_train, 
                    epochs=50, 
                    verbose=1,
                    batch_size=batch_size,
                    validation_data = (x_test, y_test),
                    callbacks=[early_stop])

Epoch 1/50
1532/1532 [==============================] - 11s 4ms/step - loss: 0.6472 - accuracy: 0.8341 - val_loss: 0.3067 - val_accuracy: 0.9130
Epoch 2/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.2689 - accuracy: 0.9250 - val_loss: 0.2376 - val_accuracy: 0.9298
Epoch 3/50
1532/1532 [==============================] - 8s 5ms/step - loss: 0.2147 - accuracy: 0.9396 - val_loss: 0.2022 - val_accuracy: 0.9401
Epoch 4/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.1819 - accuracy: 0.9489 - val_loss: 0.1773 - val_accuracy: 0.9472
Epoch 5/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.1585 - accuracy: 0.9551 - val_loss: 0.1634 - val_accuracy: 0.9531
Epoch 6/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.1411 - accuracy: 0.9600 - val_loss: 0.1492 - val_accuracy: 0.9555
Epoch 7/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.1268 - accuracy: 0.9636 - val_loss: 0.1383 - val_accuracy

Train & test accuracies are breaking the ceiling at 0.9999 & 0.9739 respectively. What a beast!

Task 2 Compare the performance of 2-layer MLP when using different activation functions.

In [10]:
# Build the model

# Define the optimizer
adam_opt = tf.keras.optimizers.Adam(learning_rate=0.0001,
                                    beta_1=0.9,
                                    beta_2=0.999)
# Instead of defining the loss, I'm going to use Softmax on the last layer
# Define the loss
# loss_ = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Define the number of units per layer
feature_size = 784
unit_size = 100
n_hidden_layers = 2

def build_mlp_model(feature_size, n_hidden_layers, unit_size, opt_, loss_):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(unit_size, activation=tf.nn.tanh, input_shape=(feature_size,)),
        tf.keras.layers.Dense(unit_size, activation=tf.nn.tanh),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer=opt_, 
                  loss=loss_,
                  metrics=['accuracy'])
    
    return model


mlp = build_mlp_model(feature_size, n_hidden_layers, unit_size, adam_opt, loss_)

mlp.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               78500     
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Start training the model using a tanh activation function

batch_size=32
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = mlp.fit(x_train, y_train, 
                    epochs=50, 
                    verbose=1,
                    batch_size=batch_size,
                    validation_data = (x_test, y_test),
                    callbacks=[early_stop])

Epoch 1/50


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5534: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1532/1532 [==============================] - 8s 4ms/step - loss: 0.6234 - accuracy: 0.8440 - val_loss: 0.3369 - val_accuracy: 0.9034
Epoch 2/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.2995 - accuracy: 0.9155 - val_loss: 0.2650 - val_accuracy: 0.9218
Epoch 3/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.2484 - accuracy: 0.9289 - val_loss: 0.2318 - val_accuracy: 0.9310
Epoch 4/50
1532/1532 [==============================] - 8s 5ms/step - loss: 0.2149 - accuracy: 0.9380 - val_loss: 0.2080 - val_accuracy: 0.9379
Epoch 5/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.1883 - accuracy: 0.9457 - val_loss: 0.1853 - val_accuracy: 0.9440
Epoch 6/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.1669 - accuracy: 0.9520 - val_loss: 0.1718 - val_accuracy: 0.9482
Epoch 7/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.1495 - accuracy: 0.9568 - val_loss: 0.1559 - val_accuracy: 0.9530
Epo

Fun Fact: you can stop training if you "can't" wait anymore.

Train accuracy: 0.9998
Test accuracy: 0.9738

We are up & running!

In [12]:
# Build the model

# Define the optimizer
adam_opt = tf.keras.optimizers.Adam(learning_rate=0.0001,
                                    beta_1=0.9,
                                    beta_2=0.999)
# Define the loss
loss_ = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Define the number of units per layer
feature_size = 784
unit_size = 100
n_hidden_layers = 2

def build_mlp_model(feature_size, n_hidden_layers, unit_size, opt_, loss_):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(unit_size, activation=tf.nn.sigmoid, input_shape=(feature_size,)),
        tf.keras.layers.Dense(unit_size, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(10)
    ])
    
    model.compile(optimizer=opt_, 
                  loss=loss_,
                  metrics=['accuracy'])
    
    return model


mlp = build_mlp_model(feature_size, n_hidden_layers, unit_size, adam_opt, loss_)

mlp.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               78500     
                                                                 
 dense_7 (Dense)             (None, 100)               10100     
                                                                 
 dense_8 (Dense)             (None, 10)                1010      
                                                                 
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Start training the model using a sigmoid activation function

batch_size=32
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = mlp.fit(x_train, y_train, 
                    epochs=50, 
                    verbose=1,
                    batch_size=batch_size,
                    validation_data = (x_test, y_test),
                    callbacks=[early_stop])

Epoch 1/50
1532/1532 [==============================] - 7s 4ms/step - loss: 1.6171 - accuracy: 0.6396 - val_loss: 1.0035 - val_accuracy: 0.8033
Epoch 2/50
1532/1532 [==============================] - 7s 4ms/step - loss: 0.7375 - accuracy: 0.8527 - val_loss: 0.5536 - val_accuracy: 0.8780
Epoch 3/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.4656 - accuracy: 0.8923 - val_loss: 0.3973 - val_accuracy: 0.8992
Epoch 4/50
1532/1532 [==============================] - 7s 5ms/step - loss: 0.3625 - accuracy: 0.9065 - val_loss: 0.3308 - val_accuracy: 0.9081
Epoch 5/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.3135 - accuracy: 0.9143 - val_loss: 0.2948 - val_accuracy: 0.9145
Epoch 6/50
1532/1532 [==============================] - 6s 4ms/step - loss: 0.2838 - accuracy: 0.9202 - val_loss: 0.2725 - val_accuracy: 0.9204
Epoch 7/50
1532/1532 [==============================] - 8s 5ms/step - loss: 0.2628 - accuracy: 0.9252 - val_loss: 0.2565 - val_accuracy:

KeyboardInterrupt: ignored

 Training accuracy is 0.9877 and test accuracy is 0.9722 after 50 epochs. Snap.